In [28]:
import os, sys, git, time, io, ntpath, tracemalloc, sqlite3,math, pandas as pd, numpy as np
from pandas.api.types import CategoricalDtype
import sklearn
from scripts.log_processing.plotting import *

from scripts.log_processing.mazeAndPcsPlotter import *
import xml.etree.ElementTree as ET


# DO NOT IMPORT IN REAL SCRIPT:
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display
from os import path
import matplotlib.collections
import matplotlib.collections as mcoll
import matplotlib.path as mpath

%matplotlib widget

In [2]:
def parse_maze(file):
    root = ET.parse(file).getroot()
    start_positions = parse_start_positions(root.find('startPositions'))
    walls = pd.concat([parse_all_walls(root), parse_all_generators(root)]).reset_index(drop=True)
    feeders = parse_all_feeders(root)

    return walls, feeders, start_positions

In [3]:
walls, feeders, start_positions = parse_maze('experiments/mazes/replayTest/M0.xml')

In [20]:
display(walls)
display(feeders)
display(start_positions)

,x1,y1,x2,y2
0,-1.1,-1.5,1.1,-1.5
1,1.1,-1.5,1.1,1.5
2,1.1,1.5,-1.1,1.5
3,-1.1,1.5,-1.1,-1.5


,fid,x,y
0,1,0.0,1.2


,x,y,w
0,0.0,-1.2,NaN


In [118]:
def genStepPaths(walls, feeders, start_positions):
    dif_x = feeders.loc[0].at['x'] - start_positions.loc[0].at['x']
    dif_y = feeders.loc[0].at['y'] - start_positions.loc[0].at['y']
    if (dif_x == 0) ^ (dif_y == 0):
        path_dist = [dif_x,'x'] if dif_y == 0 else [dif_y,'y']
        if path_dist[1] == 'x' and path_dist[0] > 0:
            direction = 'East'
        elif path_dist[1] == 'x' and path_dist[0] < 0:
            direction = 'West'
        elif path_dist[1] == 'y' and path_dist[0] < 0:
            direction = 'South'
            dist_west_wall =  start_positions.loc[0].at['x'] - walls.loc[3].at['x1']
            dist_east_wall =  walls.loc[1].at['x1'] - start_positions.loc[0].at['x']
        elif path_dist[1] == 'y' and path_dist[0] > 0:
            direction = 'North'
            dist_west_wall =  start_positions.loc[0].at['x'] - walls.loc[3].at['x1']
            dist_east_wall =  walls.loc[1].at['x1'] - start_positions.loc[0].at['x']
            min_step_to_goal = int(path_dist[0]/.08)
            section_len = math.floor(min_step_to_goal/5)
            path1_detour_steps = math.floor(dist_west_wall/.08/2)
            path2_detour_steps = math.floor(dist_west_wall/.08/2)
            
            path1_len = min_step_to_goal + (2*path1_detour_steps)
            path2_len = min_step_to_goal + (2*path2_detour_steps)
        
            north_path = 2*np.ones((section_len,), dtype=int)
            west_path = 4*np.ones((path1_detour_steps,), dtype=int)
            east_path = np.zeros((path1_detour_steps,), dtype=int)
            south_path = 6*np.ones((section_len,), dtype=int)
            
            path1 = np.concatenate([north_path,west_path,north_path,east_path,north_path,north_path,north_path])
            path2 = np.concatenate([north_path,north_path,north_path,east_path,north_path,west_path,north_path])
            
            while path1.size < path1_len:
                np.insert(path1,2)
            while path2.size < path2_len:
                np.insert(path2,2)
            
            return path1,path2
        
    
    return -1,-1
def genSpikePaths(walls, feeders, start_positions):
    dif_x = feeders.loc[0].at['x'] - start_positions.loc[0].at['x']
    dif_y = feeders.loc[0].at['y'] - start_positions.loc[0].at['y']
    if (dif_x == 0) ^ (dif_y == 0):
        path_dist = [dif_x,'x'] if dif_y == 0 else [dif_y,'y']
        if path_dist[1] == 'x' and path_dist[0] > 0:
            direction = 'East'
        elif path_dist[1] == 'x' and path_dist[0] < 0:
            direction = 'West'
        elif path_dist[1] == 'y' and path_dist[0] < 0:
            direction = 'South'
            dist_west_wall =  start_positions.loc[0].at['x'] - walls.loc[3].at['x1']
            dist_east_wall =  walls.loc[1].at['x1'] - start_positions.loc[0].at['x']
        elif path_dist[1] == 'y' and path_dist[0] > 0:
            direction = 'North'
            dist_west_wall =  start_positions.loc[0].at['x'] - walls.loc[3].at['x1']
            dist_east_wall =  walls.loc[1].at['x1'] - start_positions.loc[0].at['x']
            min_step_to_goal = int(path_dist[0]/.08)
            section_len = math.floor(min_step_to_goal/5)
            
            straight_path_steps = -1
            detour_ratio = 2
            while straight_path_steps < 0:
            
                path1_detour_steps = math.floor(dist_west_wall/.08/detour_ratio)
                path1_detour_len = math.ceil(math.sqrt(2)*path1_detour_steps)
                path2_detour_steps = math.floor(dist_west_wall/.08/detour_ratio)
                path2_detour_len = math.ceil(math.sqrt(2)*path2_detour_steps)

                straight_path_steps = min_step_to_goal - path1_detour_len - path1_detour_len
                detour_ratio -= 1
            
            straight_path_section = math.floor(straight_path_steps / 3)
            
            
            north_path = 2*np.ones((straight_path_section,), dtype=int)
            north_path_spike = 2*np.ones((path1_detour_len,), dtype=int)
            west_path = 3*np.ones((path1_detour_steps,), dtype=int)
            east_path = np.ones((path1_detour_steps,), dtype=int)
            south_path = 6*np.ones((section_len,), dtype=int)
            south_path_spike = 6*np.ones((path1_detour_len,), dtype=int)
            
            path1 = np.concatenate([north_path,west_path,east_path,north_path,north_path_spike,north_path])
            path2 = np.concatenate([north_path,north_path_spike,north_path,east_path,west_path,north_path])
            
            return path1,path2
        
    
    return -1,-1
def genLoopPaths(walls, feeders, start_positions):
    dif_x = feeders.loc[0].at['x'] - start_positions.loc[0].at['x']
    dif_y = feeders.loc[0].at['y'] - start_positions.loc[0].at['y']
    if (dif_x == 0) ^ (dif_y == 0):
        path_dist = [dif_x,'x'] if dif_y == 0 else [dif_y,'y']
        if path_dist[1] == 'x' and path_dist[0] > 0:
            direction = 'East'
        elif path_dist[1] == 'x' and path_dist[0] < 0:
            direction = 'West'
        elif path_dist[1] == 'y' and path_dist[0] < 0:
            direction = 'South'
            dist_west_wall =  start_positions.loc[0].at['x'] - walls.loc[3].at['x1']
            dist_east_wall =  walls.loc[1].at['x1'] - start_positions.loc[0].at['x']
        elif path_dist[1] == 'y' and path_dist[0] > 0:
            direction = 'North'
            dist_west_wall =  int((start_positions.loc[0].at['x'] - walls.loc[3].at['x1'])/.08/2)
            dist_east_wall =  int((walls.loc[1].at['x1'] - start_positions.loc[0].at['x'])/.08/2)
            min_step_to_goal = int(path_dist[0]/.08)
            section_len = math.floor(min_step_to_goal/5)
            
            north_path_len = math.floor((min_step_to_goal-2)/2)
            west_loop_len = math.floor((2 * dist_west_wall) + 7)
            east_loop_len = math.floor((2 * dist_east_wall) + 7)
            
            
            
            
            north_path = 2*np.ones((north_path_len,), dtype=int)
            west_loop = -1*np.ones((west_loop_len-2,), dtype=int)
            east_loop = -1*np.ones((east_loop_len-2,), dtype=int)
            
            west_loop[0:3] = 5
            west_loop[-3:] = 7
            print(len(west_loop))
            west_loop[dist_west_wall:dist_west_wall+5] = 2
            for i in range(len(west_loop)):
                if west_loop[i] == -1 and i <= dist_west_wall:
                    west_loop[i] = 4
                if west_loop[i] == -1 and i >= dist_west_wall+5:
                    west_loop[i] = 0
            print(west_loop)
            
            east_loop[0:3] = 7
            east_loop[-3:] = 5
            print(len(east_loop))
            east_loop[dist_east_wall:dist_east_wall+5] = 2
            for i in range(len(east_loop)):
                if east_loop[i] == -1 and i <= dist_east_wall:
                    east_loop[i] = 0
                if east_loop[i] == -1 and i >= dist_east_wall+5:
                    east_loop[i] = 4
            print(east_loop)
            
            path1 = np.concatenate([north_path,west_loop,north_path])
            path2 = np.concatenate([north_path,east_loop,north_path])
            
            return path1,path2
        
    
    return -1,-1

In [119]:
p1,p2 = genSpikePaths(walls, feeders, start_positions)
with open('experiments/unseen_paths/SpikePathsM0.txt', 'w') as f:
    for i in range(p1.size):
        if i == p1.size-1:
            f.write(str(p1[i])+'\n')
        else:
            f.write(str(p1[i])+',')
    for i in range(p2.size):
        if i == p2.size-1:
            f.write(str(p2[i])+'\n')
        else:
            f.write(str(p2[i])+',')
        
p1,p2 = genStepPaths(walls, feeders, start_positions)
with open('experiments/unseen_paths/StepPathsM0.txt', 'w') as f:
    for i in range(p1.size):
        if i == p1.size-1:
            f.write(str(p1[i])+'\n')
        else:
            f.write(str(p1[i])+',')
    for i in range(p2.size):
        if i == p2.size-1:
            f.write(str(p2[i])+'\n')
        else:
            f.write(str(p2[i])+',')
p1,p2 = genLoopPaths(walls, feeders, start_positions)
with open('experiments/unseen_paths/LoopPathsM0.txt', 'w') as f:
    for i in range(p1.size):
        if i == p1.size-1:
            f.write(str(p1[i])+'\n')
        else:
            f.write(str(p1[i])+',')
    for i in range(p2.size):
        if i == p2.size-1:
            f.write(str(p2[i])+'\n')
        else:
            f.write(str(p2[i])+',')

17
[5 5 5 4 4 4 2 2 2 2 2 0 0 0 7 7 7]
17
[7 7 7 0 0 0 2 2 2 2 2 4 4 4 5 5 5]


In [58]:
numpy.savetxt("file1.txt", Array)

[2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 3 3 3 3 3 3 2 2 2 2]
